# Projet phylogénétique
Marine Djaffardjy (marine.djaffardjy@u-psud.fr)- Théophile Sanchez - Sarah Cohen Boulakia

------

## Introduction

Au cours de ce projet, vous étudierez trois espèces disparues de félins qui vivaient autrefois sur le continent Américain. Ces trois espèces, le _smilodon_ (tigre à dents de sabre), l'_homotherium_ (_scimitar toothed tigers_) et _M. trumani_ (guépard américain) se sont éteintes il y a environ 13 000 ans, à la fin de la dernière période glaciaire. Des séquences ADN partielles de la protéine cytochrome b de ces espèces ont pu être séquencées et vont vous permettre de retrouver les liens de parentés entre ces espèces et des espèces de félins contemporaines : le chat domestique, le lion, le léopard, le tigre, le puma, le guépard et les chats sauvages africains, chinois et européens. Sont aussi présentes dans le jeu de donnée des séquences issues d'espèces extérieures aux félins.

Afin de reconstruire l'arbre phylogénétique de ces espèces, vous utiliserez une méthode basée sur le calcul des distances évolutives entre les séquences ADN des protéines. Sachez qu'une démarche similaire peut-être appliquée aux séquences d'acides aminés.

Les différentes étapes qui vous permetterons de construire l'arbre sont détaillées dans ce notebook. Vous devrez implémenter les algorithmes en Python et répondre aux questions dans les cellules prévues.

Quelques conseils :
- Utiliser au maximum les fonctions présentes dans les packages de python (sauf si il vous est explicitement demandé de les réimplémenter). Si un problème vous paraît courant, il existe surement déjà une fonction pour le résoudre. Pour ce projet vous serez limités aux packages de base, à Numpy et ETE (seulement pour l'affichage des arbres). Vous n'avez pas besoin d'utiliser d'autre packages (pas de pandas).
- Si une partie de votre code ne vous semble pas très explicite, ajoutez des commentaires pour l'expliquer. Une personne qui lit votre code doit pouvoir comprendre son fonctionnement facilement.
- Faites des fonctions d'affichage pour vos matrices !!
- N'hésitez pas à chercher dans la documentation et sur internet. Cependant, faites attention au plagiat !

Le projet est à rendre **en binôme** par mail. Vous regrouperez votre notebook et les fichiers nécessaires à son fonctionnement dans une archive portant vos noms et prénoms (il faut que je puisse faire tourner les notebooks).

------
## Importation des séquences

Le format [FASTA](https://fr.wikipedia.org/wiki/FASTA_(format_de_fichier) permet de stocker plusieurs séquences (ADN, ARN ou peptidiques) dans un fichier. Les séquences que vous allez étudier ont été regroupées dans le fichier `cat_dna.fasta`.

**Exercice 1 :** Écrivez une fonction permettant d'importer un fichier au format fasta et de le stocker dans un dictionnaire `seq_cat`. Les clés seront les noms des séquences et les valeurs du dictionnaire seront les séquences d'adn.
Exportez le dictionnaire obtenu dans un fichier `output/dict_seq.json`.

In [108]:
# Importe toutes les bibliothèques du projets
import json
import numpy as np

In [109]:
def load_fasta(filepath, toJson=False):
    """
    Charge un fichier Fasta à partir d'un chemin de fichier et le convertit en dictionnaire ou JSON.

    Args:
        filepath (str): Chemin d'accès au fichier Fasta
        toJson (bool, optional): Indique si le dictionnaire doit être converti en JSON. Par défaut, False.

    Returns:
        dict: dictionnaire des séquences Fasta. Si toJson est True, retourne le dictionnaire sous forme de chaîne JSON.

    Example:
        fdict = load_fasta("sequences.fasta", True)
    """
    f = open(filepath, "r")
    f_str = f.read()
    f_sections = f_str.split(">")[1:]
    f_dict = {}

    # Créer le dictionnaire
    for sect in f_sections:
        key_val = sect.split("\n")
        key = key_val[0]
        val = "".join(key_val[1:])
        f_dict[key] = val

    # Transforme en fichier JSON.
    if toJson:
        f_dict = json.dumps(f_dict)

    return f_dict

In [110]:
FASTA = load_fasta("./input/cat_dna.fasta")

print(f"Dog DNA : {FASTA['Dog DNA'][:10]}...")
print(f"Wolf DNA: {FASTA['Wolf DNA'][:10]}...")

Dog DNA : CTAGCCAAAA...
Wolf DNA: CTAGCCAAAA...


------
## Alignement des séquences

La méthode que vous utiliserez pour calculer l'arbre phylogénétique nécessite de calculer la distance évolutive entre les séquences. Avant de pouvoir les calculer, il faut d'abord aligner les séquences en considérant trois types de mutations :
- les substitutions (un nucléotide est remplacé par un autre)
- les insertions
- les délétions
Par exemple, les séquences "ACTCCTGA" et "ATCTCGTGA" ont plusieurs alignements possibles : 

$A_1$ :
```
-ACTCCTGA
ATCTCGTGA
```

$A_2$ :
```
A-CTCCTGA
ATCTCGTGA
```

$A_3$ :
```
AC-TCCTGA
ATCTCGTGA
```
.

.

.

Le "-" désigne un *gap*, c'est à dire un "trou" dans l'alignement qui a été causé par une insertion ou une déletion. On regroupe ces deux types de mutations sous le terme indel.

Ces alignements correspondent à une multitude d'histoires phylogénétiques différentes. Pour sélectionner le meilleur alignement il faut donc introduire l'hypothèse du maximum de parcimonie qui privilégie l'histoire phylogénétique qui implique le moins d'hypothèses et donc, le moins de changements évolutifs. Par exemple, parmis les trois alignements ci-dessus on preferera l'alignement 2 car il correspond au scénario avec le moins de mutations:
- l'alignement 1 implique au minimum 1 indel et 2 substitutions
- l'alignement 2 implique au minimum 1 indel et 1 substitutions
- l'alignement 3 implique au minimum 1 indel et 2 substitutions

On peut maintenant définir un score d'identité que l'on va augmenter de 1 lorsque qu'il n'y pas eu de mutation et ainsi obtenir la matrice suivante :

|   &nbsp;   | A | C | G | T | - |
|   -   | - | - | - | - | - |
| **A** | 1 | 0 | 0 | 0 | 0 |
| **C** | 0 | 1 | 0 | 0 | 0 |
| **G** | 0 | 0 | 1 | 0 | 0 |
| **T** | 0 | 0 | 0 | 1 | 0 |
| **-** | 0 | 0 | 0 | 0 | 0 |

Cette matrice correspond au modèle d'évolution de l'ADN défini par Jukes et Cantor qui fait l'hypothèse d'un taux de mutation équivalent pour chacun des nucléotides. Cependant, en réalité ces taux ne sont pas les mêmes partout, on sait par exemple que le taux de transition (substitution A$\leftrightarrow$G ou C$\leftrightarrow$T) est différent du taux de transversions (substitution A$\leftrightarrow$T, C$\leftrightarrow$G, C$\leftrightarrow$A ou G$\leftrightarrow$T) et que d'autres facteurs devrait être pris en compte comme la fréquence du nucléotide dans l'ADN. [C'est pour cette raison qu'il existe beaucoup de modèles différents d'écrivant l'évolution de l'ADN.](https://en.wikipedia.org/wiki/Models_of_DNA_evolution) Dans la suite de ce projet nous utiliserons la matrice de similarité $S$ suivante : 

|   &nbsp;   | A  | C  | G  | T  | -  |
|   -   | -  | -  | -  | -  | -  |
| **A** | 10 | -1 | -3 | -4 | -5 |
| **C** | -1 | 7  | -5 | -3 | -5 |
| **G** | -3 | -5 | 9  | 0  | -5 |
| **T** | -4 | -3 | 0  | 8  | -5 |
| **-** | -5 | -5 | -5 | -5 | 0  |

**Exercice 2 :** Écriver la fonction permettant de calculer le score entre deux alignements avec la matrice de similarité précédente puis afficher le score des trois alignements $A_1$, $A_2$ et $A_3$. La classe permettant d'importer une matrice et de calculer le score entre deux lettres vous est déjà fournie, la matrice de similarité est stockée dans le fichier `dna_matrix` :


In [111]:
class SimilarityMatrix:
    """
    Une classe pour représenter une matrice de similarité de séquences d'acides aminés

    Args:
        letters (list) : Liste des lettres représentant les acides aminés.
        values (numpy array) : Matrice de similarité, avec des scores de similarité entre les acides aminés.

    Methods:
        init(self, filename) : Initialise la matrice de similarité en lisant un fichier de similarité.
        score(self, letter1, letter2) : Retourne le score de similarité entre letter1 et letter2.
        compute_score(self, seq1, seq2) : Calcule et retourne le score total de similarité entre les séquences seq1 et seq2.
    """

    def __init__(self, filename):
        with open(filename) as f:
            self.letters = f.readline().split()
            self.values = np.loadtxt(
                filename, skiprows=1, usecols=range(1, len(self.letters) + 1))

    def score(self, letter1, letter2):  # return the similarity score between letter1 and letter2
        """
        Retourne le score de similarité entre letter1 et letter2.

        Args:
            letter1 (str) : La première lettre représentant un acide aminé à comparer.
            letter2 (str) : La deuxième lettre représentant un acide aminé à comparer.

        Returns:
            score (int) : Score de similarité entre letter1 et letter2.

        Example:
        matrix = SimilarityMatrix("similarity_matrix.txt")
        score = matrix.score("A", "A")
        """
        return self.values[self.letters.index(letter1)][self.letters.index(letter2)]

    def compute_score(self, seq1, seq2):
        """
        Calcule et retourne le score total de similarité entre les séquences seq1 et seq2.

        Args:
            seq1 (str) : La première séquence d'acides aminés à comparer.
            seq2 (str) : La deuxième séquence d'acides aminés à comparer.

        Returns:
            seq_scores (int) : Score total de similarité entre les séquences seq1 et seq2.

        Exemple:
            matrix = SimilarityMatrix("similarity_matrix.txt")
            seq_scores = matrix.compute_score("AAA", "AAA")
        """
        seq_len = len(seq1)
        seq_scores = 0
        for i in range(seq_len):
            seq_scores += self.score(seq1[i], seq2[i])

        return seq_scores


# Example
similarity_matrix = SimilarityMatrix('input/dna_matrix')
print('Score between G and C:', similarity_matrix.score('G', 'C'))
print('Score between A and a gap:', similarity_matrix.score('-', 'A'))


Score between G and C: -5.0
Score between A and a gap: -5.0


In [112]:
# Calcule le score de similarité entre les séquences suivantes.
seq = "ATCTCGTGA"
seq_1 = "-ACTCCTGA"
seq_2 = "A-CTCCTGA"
seq_3 = "AC-TCCTGA"

print('Score bettween A1:', similarity_matrix.compute_score(seq, seq_1))
print('Score bettween A2:', similarity_matrix.compute_score(seq, seq_2))
print('Score bettween A3:', similarity_matrix.compute_score(seq, seq_3))


Score bettween A1: 35.0
Score bettween A2: 49.0
Score bettween A3: 39.0


------
### Algorithme de Needleman-Wunsch

Maintenant que vous avez vu ce qu'est une matrice de similarité et comment calculer le score de similarité d'un alignement, vous allez devoir implémenter un algorithme permettant de trouver le meilleur alignement global entre deux séquences. Avec deux séquences à aligner de taille $n$ et $m$, la première étape consiste à initialiser deux matrices de taille $(n+1) \times (m+1)$. La première est la matrice de score $M$ et la seconde sera la matrice de *traceback* $T$. 

Par exemple, avec la matrice $S$ et les séquences $A =$ "ACTCCTGA" et $B =$ "ATCTCGTGA", on initialise $M$ comme si l'on ajoutait des *gaps* partout :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | 0 |-5 |-10|-15|-20|-25|-30|-35|-40|-45|
| **A** |-5 | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-10| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-15| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-20| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-25| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-30| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** |-45| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** |-40| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||

Puis on initialise $T$ :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||


Il faut ensuite remplir la matrice $M$ en suivant la formule $M_{ij} = \max(M_{i-1j-1} + s(A_i, B_j), M_{ij-1} + s(A_i, gap), M_{i-1j} + s(B_j,gap) )$ avec $i \in {2, \dots, n+1}$, $j \in {2, \dots, m+1}$ et la fonction $s$ qui calcule le score de similarité entre deux nucléotides. Pour chaque case de $T$ on remplie par :
- 'd' (*diagonal*) si $M_{ij}$ a été calculé en utilisant la diagonale $M_{i-1j-1}$,
- 'l' (*left*) si $M_{ij}$ a été calculé en utilisant la case de gauche $M_{ij-1}$,
- 'u' (*up*) si $M_{ij}$ a été calculé en utilisant la case du haut $M_{i-1j}$.

On obtient alors les matrices suivantes $M$ et $T$ : 

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** |  0| -5|-10|-15|-20|-25|-30|-35|-40|-45|
| **A** | -5| 10|  5|  0| -5|-10|-15|-20|-25|-30|
| **C** |-10|  5|  7| 12|  7|  2| -3| -8|-13|-18|
| **T** |-15|  0| 13|  8| 20| 15| 10|  5|  0| -5|
| **C** |-20| -5|  8| 20| 15| 27| 22| 17| 12|  7|
| **C** |-25|-10|  3| 15| 17| 22| 22| 19| 14| 11|
| **T** |-30|-15| -2| 10| 23| 18| 22| 30| 25| 20|
| **G** |-35|-20| -7|  5| 18| 18| 27| 25| 39| 34|
| **A** |-40|-25|-12|  0| 13| 17| 22| 23| 34| 49|

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | d | l | l | l | l | l | l | l | d |
| **C** | u | u | d | d | l | d | l | l | l | l |
| **T** | u | u | d | l | d | l | l | d | l | l |
| **C** | u | u | u | d | l | d | l | l | l | l |
| **C** | u | u | u | d | d | d | d | d | l | d |
| **T** | u | u | d | u | d | l | d | d | l | l |
| **G** | u | u | u | u | u | d | d | u | d | l |
| **A** | u | d | u | u | u | d | u | d | u | d |

Il suffit maintenant de regarder le dernier élément $M_{nm} = 49$ pour avoir le score de l'alignement. Pour avoir l'alignement lui-même, il faut partir de $T_{nm}$ et remonter la "trace" jusqu'à arriver au 'o'. Un 'd' correspond à un *match* entre les deux séquences, 'l' à un *gap* dans la séquence $A$ et 'u' à un *gap* dans la séquence $B$. En revenant à l'exemple précédent on obtient la trace suivante :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | &nbsp; | d | l | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; |
| **G** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d |

Elle correspond à l'alignement :
```
A-CTCCTGA
ATCTCGTGA
```

**Exercice 3 :** Implémenter l'algorithme de Needlman et Wunsch. Il prendra en paramètre deux séquences et une matrice de similarité et retournera leur alignement. Tester le avec les séquences "ACTCCTGA" et "ATCTCGTGA".

In [113]:
def needlmanWunsch(seq_a, seq_b, mat_similar=None):
    """
    Retourne l'alignement optimal entre seqA et seqB en utilisant l'algorithme de Needleman-Wunsch.

    Paramètres:
        seqA (str) : La première séquence à aligner.
        seqB (str) : La deuxième séquence à aligner.
        mat_similar (SimilarityMatrix) : Matrice de similarité utilisée pour évaluer la similitude entre les acides aminés.

    Returns:
        str : Alignement optimal entre seqA et seqB.

    Example:
        matrix = SimilarityMatrix("similarity_matrix.txt")
        seqA = "AGCT"
        seqB = "AACT"
        align = needlmanWunsch(seqA, seqB, similarMatrix=matrix)
    """
    # Définit le nombre de lignes et de colonnes de la matrice.
    line = len(seq_a) + 1
    col = len(seq_b) + 1

    # Récupère le score de gap.
    score_gap = int(mat_similar.score(seq_a[0], "-"))

    # Initialise les matrices M et T.
    mat_m = np.zeros((line, col))
    mat_t = np.full((line, col), " ")

    # Définit les valeurs initiaux pour la matrice M.
    mat_m[0] = np.arange(0, score_gap*col, score_gap)
    mat_m[:, :1] = np.arange(0, score_gap*line, score_gap).reshape((line, 1))

    # Définit les valeurs initiales pour la matrice T
    mat_t[0] = np.full((1, col), "l")
    mat_t[:, :1] = np.full((line, 1), "u")
    mat_t[0, 0] = "o"

    # Remplit la matrice M et T.
    for i in range(1, line):
        for j in range(1, col):
            # Calcule les valeurs suivant la formule.
            val_d = mat_m[i-1, j-1] + mat_similar.score(seq_a[i-1], seq_b[j-1])
            val_l = mat_m[i, j-1] + mat_similar.score(seq_a[i-1], "-")
            val_u = mat_m[i-1, j] + mat_similar.score(seq_b[j-1], "-")

            mat_m[i, j] = max(val_d, val_l, val_u)
            
            # Assigne les valeurs de la matrice de T.
            if mat_m[i, j] == val_d:
                mat_t[i, j] = "d"

            elif mat_m[i, j] == val_l:
                mat_t[i, j] = "l"

            elif mat_m[i, j] == val_u:
                mat_t[i, j] = "u"

    # Algorithme pour calculer l'alignement en suivant la matrice T.
    # Indice sur les cases.
    i, j = line-1, col-1

    # Indice des séquences pour calculer l'alignement.
    k_a, k_b = len(seq_a) - 1, len(seq_b) - 1
    
    cur_letter, align = "", ""
    is_end = False

    # On stope lorsqu'on rencontre une lettre 'o'.
    while not is_end:
        # On récupère la lettre de la case i, j.
        cur_letter = mat_t[i, j]

        # On compare la lettre i, j avec les lettres 'd', 'u' et 'l'.
        # On se déplace en diagonale.
        if cur_letter == "d":
            align += seq_a[k_a]
            k_a, k_b = k_a - 1, k_b - 1

        # On se déplace d'une case à droite.
        elif cur_letter == "l":
            align += "-"+seq_a[k_a]
            j -= 1

        # On se déplace d'une case en haut.
        elif cur_letter == "u":
            align += "-"+seq_b[k_b]
            i -= 1

        # On stoppe la boucle.
        else:
            is_end = True


        # On se déplace à la diagonale suivante à la prochaite étape.
        i, j = i - 1, j - 1

    return align[::-1]


In [114]:
# Calcule l'alignement des séquences suivantes :
needlmanWunsch("ACTCCTGA", "ATCTCGTGA", similarity_matrix)

'A-CTCCTGA'

----
## Matrice de distance

Dans le cas de séquences très proches, on estime que la distance évolutive réelle entre les séquences est proche de la p-distance qui est simplement le nombre de substitution dans l'alignement sur le nombre total de nucléotide. Pour simplifier, on ignore les positions alignées à des gaps. On applique ensuite la correction de Jukes-Cantor. Sa formule est $-(\frac{3}{4})\ln(1-(\frac{4}{3})\times \textit{p-distance})$.

**Exercice 4 :** Implémenter la fonction retournant la matrice de distance à partir d'un dictionnaire de séquences. 

In [115]:
def distance_sequence(seq1, seq2):
    """
    Calcule la distance de deux séquences de caractères.

    Args :
        seq1 (str) : La première séquence.
        seq2 (str) : La deuxième séquence.

    Returns :
        float : La distance entre les deux séquences.
    
    Errors:
        Les séquences seq1 et seq2 doivent avoir la même taille.
    """
    if len(seq1) != len(seq2): assert "Les séquences doivent avoir la même taille."
    
    p_distance = 0

    # Calcule la p_distance des séquences.
    for i in range(len(seq1)):
        if seq1[i] != seq2[i] and seq1[i] != "-" and seq2[i] != "-":
            p_distance += 1        

    p_distance /= len(seq1)
    return -3/4*np.log(1-4/3 * p_distance)


def mat_distance(dict_seq: dict):
    """
    Calculer la matrice de distance entre toutes les séquences dans un dictionnaire de séquences donné.

    Args:
        dictSeq (dict): Dictionnaire de séquences où les clés sont les noms des séquences 
        et les valeurs sont les séquences réelles.
    
    Returns: 
        numpy.ndarray : matrice de distance de taille n x n où n est le nombre de séquences dans le dictionnaire.
    """
    # Récupère les clefs des séquences.
    keys = np.array(list(dict_seq.keys()))
    n = len(keys) + 1
    keys = np.insert(keys, 0, f"({n}, {n})")

    # Ajoutes les indices qui corresponds au clefs.

    mat_dist = np.zeros((n, n), dtype=object)
    mat_dist[0] = keys
    mat_dist[:, 0] = keys
    
    # Remplit la matrice
    for i in range(1, n):
        for j in range(1, i):
            seq_1 = dict_seq[keys[i]]
            seq_2 = dict_seq[keys[j]]
            mat_dist[i, j] = distance_sequence(seq_1, seq_2)

    return mat_dist

In [124]:
def print_mat_dist(mat_dist):
    """
        Affiche la matrice de distances d'une taille n x n

        Args:
            mat_dist (np.ndarray) : Matrice de distances n x n
    """
    end = " | "
    n = len(mat_dist)

    def normalize_text(txt: str): return txt.ljust(8)[:8]

    # Affiche les clefs des collones
    for key_col in mat_dist[0]:
        key = normalize_text(key_col)
        print(f"{key}", end=end)

    print("\n")

    # Affiche chaque lignes
    for row in range(1, n):

        # Affiche les clefs des lignes
        key = normalize_text(mat_dist[row, 0])
        print(f"{key}", end=end)

        for col in range(1, n):
            dist_value = mat_dist[row, col]

            if dist_value == 0:
                print(f"{0:8}", end=end)

            else:
                print(f"{dist_value:.2e}", end=end)

        print("\n")


In [125]:
mat_d = mat_distance(FASTA)
print_mat_dist(mat_d)

(20, 20) | Sabertoo | Homother | American | Spotted  | Striped  | Chinese  | African  | Felis Ca | Wild Cat | Cheetah  | Puma DNA | Tiger DN | Lion DNA | Leopard  | Dog DNA  | Wolf DNA | Black Be | Brown Be | Cave Bea | 

Sabertoo |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 | 

Homother | 1.60e-01 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 | 

American | 1.60e-01 | 1.87e-01 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 | 

Spotted  | 1.47e-01 | 1.82e-01 | 1.87e-01 |        0 |        0 |        0 |        0 |        0 |        0 |   

In [ ]:
# Calcule la matrice de distance
mat_distance(FASTA)

**Exercice 4 bis :** À quoi sert la correction de Jukes-Cantor ? Quelle hypothèse fait-elle ? 

Réponse :

------
## Construction d'un arbre avec UPGMA

Grâce aux mesures de distances entre les séquences, on peut maintenant de construire l'arbre phylogénétique des globines. Vous allez devoir pour cela implémenter l'algorithme UPGMA (*unweighted pair group method with arithmetic mean*) qui, malgré son nom compliqué, est l'une des méthodes les plus simples pour la construction d'arbre.

### Le format Newick

Le format Newick est l'un des formats utilisé en phylogénie pour représenter un arbre sous la forme d'une chaine de caractère. Le principe est simple, les groupes ayant la même racine sont écrit entre parenthèses et séparés par des virgules. Un groupe peut être soit une feuille de l'arbre (dans notre cas une séquence), soit un autre groupe. La longueur de la branche de chaque groupe est écrite après un double point et l'arbre est terminé par un point virgule. Pour afficher l'arbre on peut utiliser les fonction du package ETE : 

In [ ]:
from ete3 import Tree, TreeStyle

newick_tree = '((A:2,(B:2,C:3):5):5,D:4);'
t = Tree(newick_tree)
ts = TreeStyle()
ts.show_branch_length = True
t.render('%%inline', w=183, units='mm', tree_style=ts)

**Exercice 5 :** Reécrivez l'arbre suivant au format Newick puis afficher-le. Les nombres correspondent aux longueurs des branches :
![](input/tree.png)

In [ ]:
from ete3 import Tree, TreeStyle

newick_tree = '((A:2,(B:2,C:3):5):5,D:4);'
t = Tree(newick_tree)
ts = TreeStyle()
ts.show_branch_length = True
t.render('%%inline', w=183, units='mm', tree_style=ts)

**Exercice 6 :** Expliquer la relation de parenté entre $A$, $B$ et $C$ ? Qu'elles sont les hypothèses qui pourraient expliquer ce type d'embranchement dans un arbre ? Donner une réponse détaillée.

Réponse : 

### UPGMA

L'algorithme UPGMA se base sur la matrice de distance entre les séquences. À chaque itération, les séquences avec la distance la plus faible sont regroupées puis une nouvelle matrice de distance est calculée avec le nouveau groupe. Cette étape est répétée jusqu'à n'avoir plus qu'un seul groupe. Par exemple, avec la matrice de distance entre les séquences $A$, $B$, $C$ et $D$ suivante :

|   &nbsp;   | A | B | C | D |
|   -   | - | - | - | - |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |
| **D** | 2 | 4 | 8 | &nbsp; |

Les séquences $A$ et $D$ sont les plus proches ($distance(A,D)=2$). On les regroupe et on met à jour la matrice :

|   &nbsp;   | (A, D) | B | C |
|   -   | - | - | - |
| **(A, D)** | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |

On regroupe maintenant $(A,D)$ et $B$ ($distance((A,D),B) = 4$) :

|   &nbsp;   | ((A, D), B) | C |
|   -   | - | - |
| **((A, D), B)** | &nbsp; | &nbsp; |
| **C** | 8 | &nbsp; |

Important : les nouvelles distances sont calculées en moyennant les distances entre les membres du nouveau groupe et des groupes non modifiés pondéré par le nombre d'UTOs dans chaque groupe. Avec $i$ et $j$ les deux membres du groupe nouvellement formé et k les groupes restant : $d_{ij,k} = \frac{n_id_{ik}}{n_i + n_j}+ \frac{n_jd_{jk}}{n_i + n_j}$. Par exemple avec la distance entre $((A, D), B)$ et $C$:

$distance(((A, D), B), C) = (distance((A, D), C)*2 + distance(B, C)) \mathbin{/} 3 = (8*2 + 8) \mathbin{/} 3 = 8 $.

L'arbre final écrit dans le format Newick est : $((A, D), B), C);$ 

Et avec les distances : $((A:1, D:1):1, B:2):2, C:4);$ 

**Exercice 7 :** Implémenter une version d'UPGMA qui calcule l'arbre au format Newick **avec les distances** puis appliquer votre algorithme aux données. 

In [ ]:
def upgma(matD, dictSeq: dict):
    n = matD.shape[0]
    mins = {}

    keys = np.array(list(dictSeq.keys()))

    for i in range(1, n):
        for j in range(1, i):
            x = matD[i, j]

            if np.min([x] + list(mins.keys())):
                mins[x] = {keys[i], keys[j]}

    return mins


matD = mat_distance(load_fasta("./input/cat_dna.fasta"))
upgma(matD, load_fasta("./input/cat_dna.fasta"))


**Exercice 8 :** Quelles sont les hypothèses faites par UPGMA ? Semblent-elles respectées dans le cas présent ?

Réponse : 

----
## Enracinement de l'arbre

Après avoir utilisé UPGMA pour réaliser votre arbre, l'enracinement s'est normalement fait au poids moyen. 

**Exercice 9 :** Quelle autre méthode est-il possible d'utiliser pour enraciner un arbre ? Pouvez-vous l'utilisez ici ? Si oui, afficher le nouvel arbre.

**Réponse :** 

----
## Neighbor-joining

Le neighbor-joining est un autre algorithme permettant de calculer un arbre phylogénique à partir d'une matrice de distance. Il a l'avantage de faire moins d'hypothèse qu'UPGMA sur les données (elles ne sont plus forcément ultramétrique) et il donne donc de meilleurs arbres dans presque tout les cas. Vous trouverez un example d'application de cet algorithme [ici](http://www.evolution-textbook.org/content/free/tables/Ch_27/T11_EVOW_Ch27.pdf).

**Exercice 10 :** Implémenter l'algorithme du neighbor-joining, appliquer-le aux données puis enraciner l'arbre.

In [ ]:
# À remplir

----
## Bootstrap

Le bootstrap est une méthode permettant de calculer la robustesse des branches d'un arbre. Il sagit de recréer un jeu de données artificiel en tirant n positions dans l'alignement des séquences avec remise, n étant la longueur de l'alignement. On recréer un arbre à partir de ces nouvelles données puis on le compare avec l'arbre obtenu avec les données réelles. Si une branche de l'arbre réelle est présente dans l'arbre artificiel, son bootstrap augmente de 1. On répètera la procédure 100 fois afin d'avoir un score sur 100 pour chaque branche.

**Exercice 11 :** Calculer le bootstrap des branches des arbres obtenus avec UPGMA et neighbor-joining.

In [ ]:
# À remplir

----
## Conclusion

**Exercice 12 :** Quelles sont vos conclusions par rapport à l'arbre phylogénique de _smilodon_, _homotherium_ et _M. trumani_ ? Comparer les deux méthodes. Comment expliquer les caractéristiques morphologiques similaires entre ces trois espèces ? Une réponse détaillée est attendue.

Réponse :